In [4]:
import pyodbc
import pandas as pd

In [147]:
class MainConexion:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.server_industry = r'SERVIDOR'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.database_industry = 'IPFavram'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.password_industry = '71zl6p9h'
        self.connection_string_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'
        self.connection_string_industry = f'DRIVER={{SQL Server}};SERVER={self.server_industry};DATABASE={self.database_industry};UID={self.username_solmicro};PWD={self.password_industry}'
        self.connection = None
        self.connection_industry = None
        self.tables_info = []

    def Open_Conn_Solmicro(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
        except Exception as e:
            print("Error opening connection: ", e)

    def Open_Conn_Industry(self):
        try:
            self.connection_industry = pyodbc.connect(self.connection_string_industry)
        except Exception as e:
            print("Error opening connection: ", e)

    def __del__(self):
        self.CloseConnection()

    def CloseConnection(self):
        try:
            self.connection = pyodbc.connect(self.connection_string_solmicro)
        except Exception as e:
            print("Error opening connection: ", e)

    def CloseConnectionIndustry(self):
        try:
            self.connection_industry = pyodbc.connect(self.connection_string_industry)
        except Exception as e:
            print("Error opening connection: ", e)

    def RunQuery(self,stringQuery):
        cursor = self.connection.cursor()
        registers = cursor.execute(stringQuery).fetchall()
        cursor.close()
        return registers
        # return pd.read_sql(sql=stringQuery,con=self.connection)

    def RunQueryIndustry(self,stringQuery):
        cursor = self.connection_industry.cursor()
        registers = cursor.execute(stringQuery).fetchall()
        cursor.close()
        return registers

    @staticmethod
    def get_arti_faltantes(resultArtil,result_art_provee):
        # Crear un diccionario con IDArticulo como clave y DescArticulo como valor
        artil_dict = {item[0]: item[1] for item in resultArtil}
        # Obtener solo los IDArticulo de result_art_provee
        art_provee_ids = set(item[0] for item in result_art_provee)
        # Encontrar los IDArticulo faltantes y mantener el DescArticulo correspondiente
        missing_items = [(artil_id, artil_dict[artil_id]) for artil_id in artil_dict if artil_id not in art_provee_ids]
        return missing_items
    
    @staticmethod
    def get_final_art(missing_items,result_art_provee_industry):
        artil_dict = {item[0]: item[1:] for item in result_art_provee_industry}
        art_provee_ids = set(item[0] for item in missing_items)
        result_items = [(artil_id, *artil_dict[artil_id]) for artil_id in artil_dict if artil_id  in art_provee_ids]
        return result_items
    @staticmethod
    def missing_providers(proveedores,data):
        set_data_proveedores = set(item[1] for item in data if item[1])
        set_provee = set(item[0] for item in proveedores)
        provee_dict = {item[0]: item[1] for item in proveedores}
        result_missing_proveedores = [(provee) for provee in set_data_proveedores if provee and provee not in set_provee]
        return result_missing_proveedores

    @staticmethod
    def export_to_excel(data):
        df = pd.DataFrame(data,columns=["IDArticulo","Proveedor","Plazo","Precio"])
        df.to_excel("articulos_sin_proveedor.xlsx",index=False)
        

In [148]:
i_Conexion = MainConexion()
i_Conexion.Open_Conn_Solmicro()
query_string = r"select IDArticulo, DescArticulo from tbMaestroArticulo where IDTipo= '04'"
resultArtil = i_Conexion.RunQuery(query_string)
query_string_art_prove = r"select IDArticulo from tbArticuloProveedor"
result_art_provee = i_Conexion.RunQuery(query_string_art_prove)
i_Conexion.Open_Conn_Industry()
query_string_art_industry = r"SELECT  CodigoArticulo, ProveedorHabitual, PlazoAprovisionam, PrecioCosteStandard FROM MArticulo WHERE  (ProveedorHabitual IS NOT NULL)"
result_art_provee_industry = i_Conexion.RunQueryIndustry(query_string_art_industry)
missing_items = i_Conexion.get_arti_faltantes(resultArtil,result_art_provee)
result_art = i_Conexion.get_final_art(missing_items=missing_items,result_art_provee_industry=result_art_provee_industry)
query_string_proveedores = r"select IDProveedor,DescProveedor from tbMaestroProveedor"
proveedores = i_Conexion.RunQuery(query_string_proveedores)

In [149]:
missing_provider = i_Conexion.missing_providers(proveedores,data=result_art)

In [150]:
missing_provider

['321',
 '1130',
 '1129',
 '217',
 '1111',
 '1120',
 '1139',
 '1094',
 '614',
 '1118',
 '1132',
 '163',
 '923',
 '1093',
 '31',
 '441',
 '75',
 '1128',
 '1092',
 '24',
 '1133',
 '1126',
 '1091',
 '1095',
 '1107',
 '1090',
 '1096',
 '1109']

In [109]:
i_Conexion.export_to_excel(data=result_art)